In [1]:
import re,time
import  matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus'] = False
from numpy.linalg import norm    



# data_process

In [21]:
path=r'.\input\ag_news\ag_news_gsdmm.xlsx'
data_initial=pd.read_excel(path)


vocab_max_size=10000   
vocab_min_count=5
data=data_initial['new_nltk'].values.tolist()
data_matrix=[text.split()  for text in data]
data=[ele for row in data_matrix for ele in row]

vocab =Counter(data)
vocab_arr = [[wd, vocab[wd]] for wd in vocab if vocab[wd] >vocab_min_count] 
vocab_arr = sorted(vocab_arr, key=lambda k: k[1])[::-1]   
vocab_arr = sorted(vocab_arr)              
vocab2id = {value[0]:itm for itm,value in enumerate(vocab_arr)}

#text-word
doc_word_matrix=[]
for row in data_matrix:
    temp_row=[]
    for word in row:
        if word in vocab2id:
            temp_row.append(vocab2id[word])
    doc_word_matrix.append(temp_row)

#text-word
doc_word_matrix1=[]
for row in data_matrix:
    temp_row=[]
    for word in row:
        if word in vocab2id:
            temp_row.append(word)
    doc_word_matrix1.append(temp_row)  
    
    
from sklearn.feature_extraction.text import CountVectorizer
docs = doc_word_matrix  
vocab = list(vocab2id.keys())  
n_docs = len(docs)   
n_terms = len(vocab) 
fix_seed=np.random.seed(0)
print(f'the number of docs{n_docs},the number of word{n_terms}')


# only for SeaNMF
# #PMI
# dt_mat = np.zeros([n_terms, n_terms])
# for itm in docs: 
#     for kk in itm:
#         for jj in itm:
#             if kk!=jj:
#                 dt_mat[int(kk),int(jj)] += 1.0
# D1 = np.sum(dt_mat)
# SS = D1*dt_mat
# print('SS done')
# for k in range(n_terms):
#     SS[k] /= np.sum(dt_mat[k])  
# for k in range(n_terms):
#     SS[:,k] /= np.sum(dt_mat[:,k])  
# dt_mat = [] # release memory
# SS[SS==0] = 1.0
# SS = np.maximum(np.log(SS),0.0)
# print('PPMI done',SS.shape)


#tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
doc_word_matrix_temp=['  '.join(doc) for doc in doc_word_matrix1]
tv = TfidfVectorizer()  
tv_fit = tv.fit_transform(doc_word_matrix_temp)
tfidf_matrix=pd.DataFrame(tv_fit.toarray(),columns=tv.get_feature_names_out())
print('tf-idf',tv_fit.toarray().shape)



the number of docs7409,the number of word4607
tf-idf (7409, 4607)


In [3]:
dt_mat = np.zeros([n_docs,n_terms])
for k in range(n_docs):
    for j in docs[k]:
        dt_mat[k, j] = dt_mat[k, j]+1.0
        # if k==0:
        #     print(k,'  ',j,dt_mat[k, j])

print('term doc matrix done')

term doc matrix done


# kmeans

In [41]:
from sklearn.cluster import KMeans

for j in range(2,21):
    kmeans = KMeans(n_clusters=j, random_state=42).fit(tv_fit.toarray())
    kmeans_pred=kmeans.labels_
    print('\n\n',f'Topic is {j}') 
    print(Counter(kmeans_pred))
    name='topic'+str(j)
    data_initial.insert(j+1, name, kmeans_pred.astype(int))
    print('--'*50)
data_initial.to_excel(path,index=False)




 Topic is 2
Counter({0: 976, 1: 423})
----------------------------------------------------------------------------------------------------


 Topic is 3
Counter({2: 566, 1: 447, 0: 386})
----------------------------------------------------------------------------------------------------


 Topic is 4
Counter({2: 558, 1: 399, 3: 328, 0: 114})
----------------------------------------------------------------------------------------------------


 Topic is 5
Counter({3: 543, 0: 351, 4: 243, 2: 158, 1: 104})
----------------------------------------------------------------------------------------------------


 Topic is 6
Counter({4: 413, 5: 392, 2: 306, 3: 155, 1: 96, 0: 37})
----------------------------------------------------------------------------------------------------


 Topic is 7
Counter({0: 454, 2: 284, 5: 205, 6: 166, 1: 156, 3: 97, 4: 37})
----------------------------------------------------------------------------------------------------


 Topic is 8
Counter({0: 398, 7: 249,

# NMF、SeaNMF

In [22]:
'''
Short Text Topic Modeling via SeaNMF
'''


class SeaNMF(object):
    def __init__(
        self,
        A, S,    
        IW1=[], IW2=[], IH=[],
        alpha=1.0, beta=0.1, n_topic=10, max_iter=100, max_err=1e-3, 
        rand_init=True, fix_seed=False):
        
        '''
        0.5*||A-WH^T||_F^2+0.5*alpha*||S-WW_c^T||_F^2+0.5*beta*||W||_1^2
        W = W1
        Wc = W2
        '''
        if fix_seed: 
            np.random.seed(0)
            
        self.obj=[] 
        
        self.A = A   
        self.S = S   
        self.n_row = A.shape[0]
        self.n_col = A.shape[1]
        self.n_topic = n_topic
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.B = np.ones([self.n_topic,1]) 
        self.max_err = max_err 

        if rand_init:
            self.nmf_init_rand()
        else:
            self.nmf_init(IW1, IW2, IH)
        self.nmf_iter()

    def nmf_init_rand(self):
        self.W1 = np.random.random((self.n_row, self.n_topic))
        self.W2 = np.random.random((self.n_row, self.n_topic))
        self.H = np.random.random((self.n_col, self.n_topic))
        print(self.W1.shape,self.W2.shape,self.H.shape,self.A.shape)

        for k in range(self.n_topic):
            self.W1[:, k] /= norm(self.W1[:, k])    # np.linalg.norm
            self.W2[:, k] /= norm(self.W2[:, k])

    def nmf_init(self, IW1, IW2, IH):
        self.W1 = IW1
        self.W2 = IW2
        self.H = IH

        for k in range(self.n_topic):
            self.W1[:, k] /= norm(self.W1[:, k])
            self.W2[:, k] /= norm(self.W2[:, k])


    def nmf_iter(self):
        loss_old = 1e20
        print('loop begin')
        start_time = time.time()
        for i in range(self.max_iter):
            self.nmf_solver() 
            loss = self.nmf_loss()  
#             print(i,' ',loss)
            self.obj.append(loss)
            if loss_old-loss < self.max_err:  #  0.001
                break
            loss_old = loss
            end_time = time.time()
            print('Step={}, Loss={}, Time={}s'.format(i, loss, end_time-start_time))
            
    def nmf_solver(self):
        '''
        using BCD framework
        '''
        epss = 1e-20
        # Update W1
        AH = np.dot(self.A, self.H)  
        SW2 = np.dot(self.S, self.W2)  
        HtH = np.dot(self.H.T, self.H) 
        W2tW2 = np.dot(self.W2.T, self.W2)
        W11 = self.W1.dot(self.B)
#         print(sum(AH),sum(SW2),sum(HtH),sum(W2tW2),sum(W11))
        for k in range(self.n_topic):
            num0 = HtH[k,k]*self.W1[:,k] + self.alpha*W2tW2[k,k]*self.W1[:,k]
            num1 = AH[:,k] + self.alpha*SW2[:,k]
            num2 = np.dot(self.W1, HtH[:,k]) + self.alpha*np.dot(self.W1, W2tW2[:,k]) + self.beta*W11[0]
#             print('---------',num2.shape,W11.shape,W11[0].shape)
            self.W1[:,k] = num0 + num1 - num2
            self.W1[:,k] = np.maximum(self.W1[:,k], epss) # project > 0
            self.W1[:,k] /= norm(self.W1[:,k]) + epss # normalize
            
            
        # Update W2
        W1tW1 = self.W1.T.dot(self.W1)
        StW1 = np.dot(self.S, self.W1)
        for k in range(self.n_topic):
            self.W2[:,k] = self.W2[:,k] + StW1[:,k] - np.dot(self.W2, W1tW1[:,k])
            self.W2[:,k] = np.maximum(self.W2[:,k], epss)
        #Update H
        AtW1 = np.dot(self.A.T, self.W1)
        for k in range(self.n_topic):
            self.H[:,k] = self.H[:,k] + AtW1[:,k] - np.dot(self.H, W1tW1[:,k])
            self.H[:,k] = np.maximum(self.H[:,k], epss) 
            
            

    def nmf_loss(self):
        '''
        Calculate loss
        '''
        loss = norm(self.A - np.dot(self.W1, np.transpose(self.H)), 'fro')**2/2.0
        if self.alpha > 0:
            loss += self.alpha*norm(np.dot(self.W1, np.transpose(self.W2))-self.S, 'fro')**2/2.0
        if self.beta > 0:
            loss += self.beta*norm(self.W1, 1)**2/2.0
            
        return loss
    def return_loss(self):
        return self.obj
    
    def return_weight(self):
        return self.W1, self.W2, self.H
    
        
        
        
'''
Topic Modeling via NMF
'''
class NMF(object):   
    def __init__(
        self, 
        A, n_topic,IW=[], IH=[],
         max_iter=100, max_err=1e-3,
        rand_init=True):
        '''
        A = WH^T
        '''
        self.A = A   #  input
        self.n_row = A.shape[0]
        self.n_col = A.shape[1]
        self.n_topic = n_topic
        self.max_iter = max_iter
        self.max_err = max_err

        self.obj = []
        if rand_init:
            self.nmf_init_rand()
        else:
            self.nmf_init(IW, IH)
        self.nmf_iter()

    def nmf_init_rand(self):
        self.W = np.random.random((self.n_row, self.n_topic))
        self.H = np.random.random((self.n_col, self.n_topic))

        print('nmf_init_rand')
        for k in range(self.n_topic):
            self.W[:,k] /= norm(self.W[:,k])
        print(self.W.shape,self.H.shape)

    def nmf_init(self, IW, IH):
        self.W = IW
        self.H = IH
        print('nmf_init')
        # for k in range(self.n_topic):
        #     self.W[:,k] /= norm(self.W[:,k])
    def nmf_iter(self):
        loss_old = 1e20
        start_time = time.time()         
        for i in range(self.max_iter):
            self.nmf_solver()    
            loss = self.nmf_loss()   
            self.obj.append(loss)
            if loss_old-loss < self.max_err:
                break
            loss_old = loss
            end_time = time.time()
            print('iter:{}, loss:{}, time{}s'.format(i, round(loss,5), end_time-start_time))            
    def nmf_solver(self):
        '''
        regular NMF without constraint.
        Block Coordinate Decent
        '''
        epss = 1e-20       
      
        HtH = self.H.T.dot(self.H) 
        AH = self.A.dot(self.H)  
#         H:(15001, 10) A:(2443, 15001) HTH(10, 10) AH:(2443, 10) W:(2443, 10)
#         print(f'H:{self.H.shape} A:{self.A.shape} HTH{HtH.shape} AH:{AH.shape} W:{self.W.shape}')
        for k in range(self.n_topic):
            tmpW = self.W[:,k]*HtH[k,k] + AH[:,k] - np.dot(self.W, HtH[:,k])
            self.W[:,k] = np.maximum(tmpW, epss)
            self.W[:,k] /= norm(self.W[:,k]) + epss
        WtW = self.W.T.dot(self.W)
        AtW = self.A.T.dot(self.W)
        for k in range(self.n_topic):
            self.H[:,k] = self.H[:,k]*WtW[k,k] + AtW[:,k] - np.dot(self.H, WtW[:,k])
            self.H[:,k] = np.maximum(self.H[:,k], epss)
    def nmf_loss(self):
        loss = norm(self.A - np.dot(self.W, self.H.T), 'fro')**2/2.0
        return loss 
    def get_loss(self):
        return np.array(self.obj) 
    def return_weight(self):
        return self.W,self.H

#  NMF long text

In [33]:
X=tv_fit.toarray().T  
max_iter=100
max_err=1e-3


In [34]:
for j in range(2,21):  #topics
    H_nnsvd, W_nnsvd = nndsvd(tv_fit.toarray(), j)
    # print(W_nnsvd.shape,H_nnsvd.shape)
    model = NMF(
        X, 
        n_topic=j,
        max_iter=max_iter, 
        max_err=max_err,rand_init=False, IW=W_nnsvd.T,IH=H_nnsvd)

    model.nmf_iter()
    W,H=model.return_weight()
    nmf_pred=np.zeros(H.shape[0])
    for index,value in enumerate(H):
        nmf_pred[index]=list(value).index(np.max(value))
    print('\n\n',f'Topic is {j}') 
    print(Counter(nmf_pred))
    name='topic'+str(j)
    data_initial.insert(j+1, name, nmf_pred.astype(int))
    print('--'*50)
data_initial.to_excel(path,index=False)

nmf_init
iter:0, loss:350.64707, time0.025780677795410156s
iter:1, loss:349.74896, time0.062027931213378906s
iter:2, loss:349.5364, time0.08876776695251465s
iter:3, loss:349.44942, time0.11713027954101562s
iter:4, loss:349.41395, time0.1458578109741211s
iter:5, loss:349.39868, time0.1739215850830078s
iter:6, loss:349.39171, time0.20156240463256836s
iter:7, loss:349.38829, time0.23219013214111328s
iter:8, loss:349.3865, time0.263073205947876s
iter:9, loss:349.38549, time0.29103517532348633s
iter:0, loss:349.38455, time0.03334355354309082s


 Topic is 2
Counter({0.0: 486, 1.0: 251})
----------------------------------------------------------------------------------------------------
nmf_init
iter:0, loss:345.78235, time0.03145432472229004s
iter:1, loss:344.7011, time0.05091047286987305s
iter:2, loss:344.37817, time0.0874643325805664s
iter:3, loss:344.23952, time0.11754894256591797s
iter:4, loss:344.1785, time0.14668560028076172s
iter:5, loss:344.14826, time0.16773319244384766s
iter:6, los

In [35]:
data_initial.to_excel(path,index=False)


# SeaNMF short text 

In [149]:
max_iter=100
# n_topics=5
alpha=1.0
beta=0.1
max_err=1e-3
fix_seed=np.random.seed(0)
X=tv_fit.toarray().T



In [145]:
for j in range(2,21):  #topics
    model = SeaNMF(
        X, SS,  
        alpha=alpha, 
        beta=beta, 
        n_topic=j, 
        max_iter=max_iter, 
        max_err=max_err,
        fix_seed=fix_seed)
    
    W1,W2,H=model.return_weight()
    print(W1.shape,W2.shape,H.shape)
    
    seanmf=np.zeros(H.shape[0])
    for index,value in enumerate(H):
        seanmf[index]=list(value).index(np.max(value))
    print('\n\n',f'Topic is {j}') 
    print(Counter(seanmf))
    name='topic'+str(j)
    data_initial.insert(j+1, name, seanmf.astype(int))
    print('--'*50)

(4607, 5) (4607, 5) (7409, 5) (4607, 7409)
loop begin
Step=0, Loss=2984323.0999932904, Time=1.1982970237731934s
Step=1, Loss=2948516.1537592267, Time=2.2294046878814697s
Step=2, Loss=2921080.8203444136, Time=3.1541669368743896s
Step=3, Loss=2897106.5617011855, Time=4.0312864780426025s
Step=4, Loss=2884448.6208720887, Time=4.910534381866455s
Step=5, Loss=2877602.9145164425, Time=5.772297620773315s
Step=6, Loss=2874438.7829573588, Time=6.613667726516724s
Step=7, Loss=2872999.65184529, Time=7.501529932022095s
Step=8, Loss=2872274.1346848207, Time=8.454342603683472s
Step=9, Loss=2871879.002158072, Time=9.311069011688232s
Step=10, Loss=2871650.8148790365, Time=10.202889680862427s
Step=11, Loss=2871510.6369883013, Time=11.125685214996338s
Step=12, Loss=2871419.394813475, Time=12.030350685119629s
Step=13, Loss=2871356.89767605, Time=12.872650861740112s
Step=14, Loss=2871312.0473810756, Time=13.687061071395874s
Step=15, Loss=2871278.6100794934, Time=14.575299978256226s
Step=16, Loss=2871253.03

ValueError: cannot insert topic5, already exists

In [151]:
data_initial.to_excel(path,index=False)

# LDA long text

In [33]:
from sklearn.decomposition import LatentDirichletAllocation as LDA


In [37]:
matrix=dt_mat
for j in range(2,21):
    lda=LDA(n_components=j, 
             max_iter=50,
             learning_method='batch',    
             learning_offset=50,
             # doc_topic_prior=0.1,
             # topic_word_prior=0.01,
             random_state=0)
    lda.fit(matrix)
    topics=lda.transform(matrix)  
    topic=[]
    for i in topics:
        topic.append(list(i).index(np.max(i)))  
    print('\n\n',f'Topic is {j}') 
    print(Counter(topic))
    name='topic'+str(j)
    data_initial.insert(j+1, name, topic)
    print('--'*50)



 Topic is 4
Counter({1: 427, 2: 400, 0: 313, 3: 259})
----------------------------------------------------------------------------------------------------


In [39]:
data_initial.to_excel(path,index=False)

# GSDMM short text

In [4]:
#gsdmm
from numpy.random import multinomial
from numpy import log, exp
from numpy import argmax
import json

class MovieGroupProcess:
    def __init__(self, K=8, alpha=0.1, beta=0.1, n_iters=30):
        '''
        Reference: http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf
        
        :param K: int
            Upper bound on the number of possible clusters. Typically many fewer
        :param alpha: float between 0 and 1
            Alpha controls the probability that a student will join a table that is currently empty
            When alpha is 0, no one will join an empty table.
        :param beta: float between 0 and 1
            Beta controls the student's affinity for other students with similar interests. A low beta means
            that students desire to sit with students of similar interests. A high beta means they are less
            concerned with affinity and are more influenced by the popularity of a table
        :param n_iters:
        '''
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.n_iters = n_iters

        # slots for computed variables
        self.number_docs = None
        self.vocab_size = None
        self.cluster_doc_count = [0 for _ in range(K)]
        self.cluster_word_count = [0 for _ in range(K)]
        self.cluster_word_distribution = [{} for i in range(K)]

    @staticmethod
    def _sample(p):
        '''
        Sample with probability vector p from a multinomial distribution
        :param p: list
            List of probabilities representing probability vector for the multinomial distribution
        :return: int
            index of randomly selected output
        '''
        return [i for i, entry in enumerate(multinomial(1, p)) if entry != 0][0]

    def fit(self, docs, vocab_size):
        '''
        Cluster the input documents
        :param docs: list of list
            list of lists containing the unique token set of each document
        :param V: total vocabulary size for each document
        :return: list of length len(doc)
            cluster label for each document
        '''
        alpha, beta, K, n_iters, V = self.alpha, self.beta, self.K, self.n_iters, vocab_size

        D = len(docs)
        self.number_docs = D
        self.vocab_size = vocab_size

        # unpack to easy var names
        # mz number of documents in cluster z 
        # nz number of words in cluster z   
        # nwz number of occurrences of word w in cluster z  
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution
        cluster_count = K

        d_z = [None for i in range(len(docs))]
        
        # initialize the clusters

        for i, doc in enumerate(docs):
            # choose a random  initial cluster for the doc
            z = self._sample([1.0 / K for _ in range(K)])
            d_z[i] = z  
            m_z[z] += 1  
            n_z[z] += len(doc)
            
            for word in doc:
                if word not in n_z_w[z]:
                    n_z_w[z][word] = 0  
                n_z_w[z][word] += 1   
        # print(n_z_w[0])
        # print(n_z[0])
        for _iter in range(n_iters):
            total_transfers = 0
            for i, doc in enumerate(docs):
                
                # remove the doc from it's current cluster
                z_old = d_z[i]
                m_z[z_old] -= 1 
                n_z[z_old] -= len(doc)
                for word in doc:

                    n_z_w[z_old][word] -= 1   
                    # compact dictionary to save space
                    if n_z_w[z_old][word] == 0:  
                        del n_z_w[z_old][word]

                # draw sample from distribution to find new cluster
                p = self.score(doc)
                z_new = self._sample(p)  

                # transfer doc to the new cluster
                if z_new != z_old:
                    total_transfers += 1
                #先减后加
                d_z[i] = z_new
                m_z[z_new] += 1
                n_z[z_new] += len(doc)
                for word in doc:
                    if word not in n_z_w[z_new]:
                        n_z_w[z_new][word] = 0
                    n_z_w[z_new][word] += 1

            cluster_count_new = sum([1 for v in m_z if v > 0])
            print("In stage %d: transferred %d clusters with %d clusters populated" % (
            _iter, total_transfers, cluster_count_new))
            if total_transfers == 0 and cluster_count_new == cluster_count and _iter>25:
                print("Converged.  Breaking out.")
                break
            cluster_count = cluster_count_new
        self.cluster_word_distribution = n_z_w
        return d_z

    def score(self, doc):
        '''
        Score a document

        Implements formula (3) of Yin and Wang 2014.
        http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf

        :param doc: list[str]: The doc token stream
        :return: list[float]: A length K probability vector where each component represents
                              the probability of the document appearing in a particular cluster
        '''
        alpha, beta, K, V, D = self.alpha, self.beta, self.K, self.vocab_size, self.number_docs
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution
        p = [0 for _ in range(K)]
        #  We break the formula into the following pieces
        #  p = N1*N2/(D1*D2) = exp(lN1 - lD1 + lN2 - lD2)
        #  lN1 = log(m_z[z] + alpha)
        #  lN2 = log(D - 1 + K*alpha)
        #  lN2 = log(product(n_z_w[w] + beta)) = sum(log(n_z_w[w] + beta))
        #  lD2 = log(product(n_z[d] + V*beta + i -1)) = sum(log(n_z[d] + V*beta + i -1))

        lD1 = log(D - 1 + K * alpha)
        doc_size = len(doc)
        for label in range(K):   
            lN1 = log(m_z[label] + alpha)  
            lN2 = 0
            lD2 = 0
            for word in doc:
                lN2 += log(n_z_w[label].get(word, 0) + beta) 
            for j in range(1, doc_size +1):
                lD2 += log(n_z[label] + V * beta + j - 1)     
            p[label] = exp(lN1 - lD1 + lN2 - lD2)

        # normalize the probability vector
        pnorm = sum(p)
        pnorm = pnorm if pnorm>0 else 1
        return [pp/pnorm for pp in p]

    def choose_best_label(self, doc):
        '''
        Choose the highest probability label for the input document
        :param doc: list[str]: The doc token stream
        :return:
        '''
        p = self.score(doc)
        return argmax(p),max(p)

In [22]:
import random
for j in range(2,21):
    gsdmm = MovieGroupProcess(K=j, alpha=0.1, beta=0.1, n_iters=30)
    topic = gsdmm.fit(data_matrix, len(vocab_arr))
    print('\n\n',f'Topic is {j}') 
    print(Counter(topic))
    name='topic'+str(j)
    # data_initial.insert(j+1, name, topic)
    print('--'*50)

In stage 0: transferred 4691 clusters with 4 clusters populated
In stage 1: transferred 2619 clusters with 4 clusters populated
In stage 2: transferred 1277 clusters with 4 clusters populated
In stage 3: transferred 697 clusters with 4 clusters populated
In stage 4: transferred 425 clusters with 4 clusters populated
In stage 5: transferred 309 clusters with 4 clusters populated


KeyboardInterrupt: 

In [25]:
data_initial.to_excel(path,index=False)

# NNDSVD

In [17]:

# functions to extract positive and negative part
def pos(vec):
  return np.array(list(map(lambda x: max(x, 0), vec)))

def neg(vec):
  return np.array(list(map(lambda x: max(-x, 0), vec)))

def normalize(vec):
  return vec / np.linalg.norm(vec)

def nndsvd(A, k):
  # here we will assume k < rank(A)
  # Get shape of A
  m, n = A.shape

  # conduct SVD
  U, S, V = np.linalg.svd(A, full_matrices = False)

  # store resulting matrix
  W, H = np.zeros((m, k)), np.zeros((k, n))

  # update rows and columns of W and H
  for i in range(k):
    if i == 0:
      W[:, 0] = np.sqrt(S[0]) * np.abs(U[:, 0])
      H[0, :] = np.sqrt(S[0]) * np.abs(V[0, :])
    else:
      x = U[:, i]
      y = V[i, :]
      xp, xn = pos(x), neg(x)
      yp, yn = pos(y), neg(y)
      if np.linalg.norm(xp) * np.linalg.norm(yp) > np.linalg.norm(xn) * np.linalg.norm(yn):
        u = normalize(xp)
        v = normalize(yp)
        sigma = np.linalg.norm(xp) * np.linalg.norm(yp)
      else:
        u = normalize(xn)
        v = normalize(yn)
        sigma = np.linalg.norm(xn) * np.linalg.norm(yn)
      W[:, i] = np.sqrt(S[i] * sigma) * u
      H[i, :] = np.sqrt(S[i] * sigma) * v
  return W, H
W_nnsvd, H_nnsvd = nndsvd(tv_fit.toarray(), k)

In [18]:
W_nnsvd.shape,H_nnsvd.shape

((737, 4), (4, 3272))

In [62]:
W_nnsvd= np.maximum(W_nnsvd,1e-6)
H_nnsvd= np.maximum(H_nnsvd,1e-6)
